In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"

import sys
import time
import datetime
import argparse
import tensorboardX
import torch
import yaml
import shutil
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
import torch.nn as nn
sys.path.append('../')

from pycode.dataset import RLBench_dataset_test, Softargmax_dataset_test
from pycode.config import _C as cfg
from pycode.model.Hourglass import sequence_hourglass
from pycode.loss.mp_loss import Test_Loss_sequence_hourglass
from pycode.misc import save_outputs, build_model_MP, build_dataset_MP, str2bool, save_args, save_checkpoint, load_checkpoint, load_hourglass

# parser
parser = argparse.ArgumentParser(description='parser for image generator')
parser.add_argument('--config_file', type=str, default='', metavar='FILE', help='path to config file')
parser.add_argument('--output_dirname', type=str, default='', help='')
parser.add_argument('--checkpoint_path', type=str, default=None, help='')
parser.add_argument('--log2wandb', type=str2bool, default=True)
parser.add_argument('--wandb_group', type=str, default='')
args = parser.parse_args(args=['--config_file', '../configs/RLBench_MP.yaml',
                                '--checkpoint_path','../output/RLBench/base_hourglass/model_log/checkpoint_iter20000/',
                                '--output_dirname','hoge','--log2wandb', 'False'])
# args = parser.parse_args()

kornia requires version >= 3.6. your version 3.6


In [2]:
# get cfg data
if len(args.config_file) > 0:
    print('Loaded configration file {}'.format(args.config_file))
    cfg.merge_from_file(args.config_file)

# define output dirname
if len(args.output_dirname) == 0:
    dt_now = datetime.datetime.now()
    output_dirname = str(dt_now.date()) + '_' + str(dt_now.time())
else:
    output_dirname = args.output_dirname
    
output_dirname = os.path.join(cfg.BASIC.OUTPUT_DIR, cfg.DATASET.NAME, output_dirname)
cfg.freeze()

# define save model path
save_path = os.path.join(args.checkpoint_path, 'test')

# make output dir
os.makedirs(output_dirname, exist_ok=True)
os.makedirs(save_path, exist_ok=True)

# copy config file
if len(args.config_file) > 0:
    shutil.copy(args.config_file,output_dirname)

# save args
argsfile_path = os.path.join(output_dirname, "args.txt")
save_args(args,argsfile_path)

# set seed and cuda
torch.manual_seed(cfg.BASIC.SEED)
cuda = torch.cuda.is_available()
device = torch.device(cfg.BASIC.DEVICE)

if cuda:
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = True
    torch.cuda.manual_seed(cfg.BASIC.SEED)

# set wandb
with open(args.config_file) as file:
    obj = yaml.safe_load(file)

if args.log2wandb:
    import wandb
    wandb.login()
    if args.wandb_group == '':
        group = None
    else:
        group = args.wandb_group
    run = wandb.init(project='MotionPrediction-{}-test'.format(cfg.DATASET.NAME), entity='tendon',
                    config=obj, save_code=True, name=args.output_dirname, dir=os.path.join(cfg.BASIC.OUTPUT_DIR, cfg.DATASET.NAME),
                    group=group)

# set dataset
if cfg.DATASET.NAME == 'HMD':
    test_dataset = Softargmax_dataset_test(cfg, save_dataset=True)
elif cfg.DATASET.NAME == 'RLBench':
    test_dataset = RLBench_dataset_test(cfg, save_dataset=True)
else:
    raise ValueError("Invalid dataset name")

# set dataloader
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=cfg.BASIC.WORKERS)

# set model
if cfg.DATASET.NAME == 'HMD':
    output_dim = 21
elif cfg.DATASET.NAME == 'RLBench':
    output_dim = 1

model = sequence_hourglass(cfg, output_dim=output_dim)
model_path = os.path.join(args.checkpoint_path, 'mp.pth')
if cfg.MP_MODEL_NAME == 'hourglass':
    model.hour_glass, _, _, _, _ = load_checkpoint(model.hour_glass, model_path, fix_parallel=True)
elif cfg.MP_MODEL_NAME == 'sequence_hourglass':
    model, _, _, _, _ = load_checkpoint(model, checkpoint_path, fix_parallel=True)

model = model.to(device)

# set loss
test_loss = Test_Loss_sequence_hourglass(cfg, device)

Loaded configration file ../configs/RLBench_MP.yaml
length of future is 1 frame
There is no json data
create json data
done
save json data
done
fix parallel


In [2]:
# start train
for iteration, inputs in enumerate(test_dataloader, 1):
    
    with torch.no_grad():
        outputs = model(inputs,mp_mode=1,vp_mode=2)
        _ = test_loss(inputs, outputs)
    
    # save_outputs(inputs, outputs, save_path, iteration, cfg, mode='test')
    print("{} / {}".format(iteration, len(test_dataloader)))

# save and print log
log = train_loss.get_log()
for key in log.keys():
    print('key:{}  Value:{}'.format(key, log[key]))

if args.log2wandb:
    wandb.log(log,step=total_iteration)

1 / 500


Process Process-4:
Process Process-16:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 261, in _bootstrap
    util._exit_function()
Traceback (most recent call last):
Process Process-14:
  File "/usr/lib/python3.6/multiprocessing/util.py", line 322, in _exit_function
    _run_finalizers()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 261, in _bootstrap
    util._exit_function()
  File "/usr/lib/python3.6/multiprocessing/util.py", line 262, in _run_finalizers
    finalizer()
  File "/usr/lib/python3.6/multiprocessing/util.py", line 322, in _exit_function
    _run_finalizers()
  File "/usr/lib/python3.6/multiprocessing/util.py", line 186, in __call__
    res = self._callback(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/util.py", line 262, in _run_finalizers
    finalizer()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 191, in _finalize_join
    thread

Traceback (most recent call last):
  File "/misc/dl00/dl001/tendon/project/Imitation_Learning/MotionPrediction/env/dl21/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-06e1168a0aec>", line 5, in <module>
    outputs = model(inputs,mp_mode=1,vp_mode=2)
  File "/misc/dl00/dl001/tendon/project/Imitation_Learning/MotionPrediction/env/dl21/local/lib/python3.6/site-packages/torch/nn/modules/module.py", line 727, in _call_impl
    result = self.forward(*input, **kwargs)
  File "../pycode/model/Hourglass.py", line 328, in forward
    outputs = self.hour_glass(model_inputs)
  File "/misc/dl00/dl001/tendon/project/Imitation_Learning/MotionPrediction/env/dl21/local/lib/python3.6/site-packages/torch/nn/modules/module.py", line 727, in _call_impl
    result = self.forward(*input, **kwargs)
  File "../pycode/model/Hourglass.py", line 188, in forward
    uv, argmax, heatmap_softm

TypeError: object of type 'NoneType' has no len()